# How to deploy ADK Apps to Agent Engine

[Documentation link 🔗](https://google.github.io/adk-docs/deploy/agent-engine/)

## 1. Set the local environment variables
These will be used in the Agent Engine deployment

In [1]:
from dotenv import load_dotenv
import os

load_dotenv("../trends_and_insights_agent/.env")  # take environment variables

True

## 2. Set up the `AdkApp` Convenience wrapper for ADK Agents -> Agent Engine

Note the injection of the environment variables.

In [2]:
from vertexai.preview.reasoning_engines import AdkApp

from trends_and_insights_agent import agent

env_vars = {}

env_vars["GOOGLE_CLOUD_PROJECT"] = os.getenv("GOOGLE_CLOUD_PROJECT")
env_vars["GOOGLE_CLOUD_LOCATION"] = os.getenv("GOOGLE_CLOUD_LOCATION")
env_vars["GOOGLE_GENAI_USE_VERTEXAI"] = os.getenv("GOOGLE_GENAI_USE_VERTEXAI")
env_vars["BUCKET"] = os.getenv("BUCKET")
env_vars["GOOGLE_CLOUD_PROJECT_NUMBER"] = os.getenv("GOOGLE_CLOUD_PROJECT_NUMBER")
env_vars["YT_SECRET_MNGR_NAME"] = os.getenv("YT_SECRET_MNGR_NAME")


my_agent = AdkApp(agent=agent.root_agent, enable_tracing=False, env_vars=env_vars)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
Invalid config for agent insights_generator_agent: output_schema cannot co-exist with agent transfer configurations. Setting disallow_transfer_to_parent=True, disallow_transfer_to_peers=True
/Users/jwortz/zghost/trends_and_insights_agent/common_agents/ad_content_generator/tools.py:120: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
  if existing_image_filename is not "":
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
Invalid config for agent trends_generator_agent: output_schema cannot co-exist with agent transfer configurations. Setting disallow_transfer_to_parent=True, disallow_transfer_to_peers=True


# 3. Test the Agent Engine locally

In [14]:
session = my_agent.create_session(user_id="jwortz")
session

Session(id='29d48183-1766-4f09-ac36-6600f70ce2b7', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1746391474.237503)

In [15]:
my_agent.list_sessions(user_id="jwortz")

ListSessionsResponse(sessions=[Session(id='29d48183-1766-4f09-ac36-6600f70ce2b7', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1746391474.237503)])

In [16]:
EXAMPLE_BRIEF = """
Campaign Name: Connect Beyond Limits with Pix Phone 5
Campaign Objectives
● Increase awareness of the latest model of the Pix Phone
● Generate leads and drive phone sales
● Position Pix Phone and the trendy phone to have
Target Audience
● Individuals aged 20-40 in major markets such as US, France, Japan
Media Strategy
1. Social Media Marketing: Run targeted social media ads on platforms where the
target audience is active.
2. Influencer Marketing: Partner with influencers in the tech industry to promote Pix
Phone 5
3. Paid Advertising: Run targeted display ads on websites and apps frequented by
the target audience. Use search engine marketing (SEM) to bid on relevant
keywords and appear in search results when potential customers are looking for
asset protection insurance.
Timeline and country
● Activity in the 3 major markets in at least the online channels by early Oct 2023
● Start from US, followed by France then Japan
● The campaign should use the US version as anchor and localized accordingly
Performance Metrics
1. Track website conversion, lead generation, and phone sales to measure campaign
effectiveness.
2. Use social media analytics to monitor engagement, reach, and sentiment.
"""

In [17]:
from pprint import pprint


def stream_agent(agent, prompt: str) -> None:
    for event in agent.stream_query(
        user_id="jwortz",
        session_id=session.id,
        message=prompt,
    ):
        pprint(event)

In [18]:
first_prompt = f"Check out the {EXAMPLE_BRIEF} and come up with some good campaign ideas. Be sure to generate a few image ideas for the campaign."
stream_agent(my_agent, first_prompt)

INFO:google_genai.models:AFC is enabled with max remote calls: 10.


INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/wortz-project-352116/locations/us-central1/publishers/google/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
INFO:root:
[Callback] Entering agent: marketing_idea_generator_agent (Inv: e-1ba86531-93e3-4b5c-b829-1ab172c55269)
INFO:root:[Callback] Current State: {}


{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'marketing_idea_generator_agent',
 'content': {'parts': [{'text': "Hello! I'm here to help you brainstorm some "
                                'great campaign ideas for the "Connect Beyond '
                                'Limits with Pix Phone 5" campaign.\n'
                                '\n'
                                'To start, could you please provide the '
                                'marketing campaign guide? You can paste the '
                                'text, provide a URL to a webpage, or upload a '
                                'PDF. This will help me get a better '
                                'understanding of the campaign and your '
                                'goals.\n'
                                '\n'
                                'For example, you could say:\n'
                                '\n'
                 

In [19]:
followup_prompt = "Create images from these great ideas!"
stream_agent(my_agent, followup_prompt)

INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/wortz-project-352116/locations/us-central1/publishers/google/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
INFO:root:
[Callback] Entering agent: marketing_idea_generator_agent (Inv: e-3b3a9ab5-22b9-44c3-9e79-ea5c1ed02ccc)
INFO:root:[Callback] Current State: {'campaign_guide': {'campaign_guide': 'not yet populated'}, 'trends': {'trends': []}, 'insights': {'insights': []}}


{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'marketing_idea_generator_agent',
 'content': {'parts': [{'text': 'I am sorry, I need the campaign guide first '
                                'before generating images. Please provide the '
                                'campaign guide so I can assist you '
                                'further.\n'}],
             'role': 'model'},
 'id': 'AbgFoXPI',
 'invocation_id': 'e-3b3a9ab5-22b9-44c3-9e79-ea5c1ed02ccc',
 'timestamp': 1746391477.98363}


## 4. Deploy to Vertex AI Agent Engine
To call and manage agents in production, deploy the agent to Vertex AI Agent Engine.

**Important - run `poetry build` to package the agent**

In [20]:
! poetry build --format=wheel --output=notebooks/deployment

Building trends_and_insights_agent (0.1.0)
  - Building wheel
  - Built trends_and_insights_agent-0.1.0-py3-none-any.whl


#### Initialize the Vertex client, then create a `remote_agent` that is deployed to Vertex

This also takes the packaged agent code. This is required for more complex agents that have nested dependencies and require packaging

Also, before running - be sure to give Secret Manager access to the Agent Engine service account. This can simply be done by running `. setup_ae_sm_access.sh`.

The code can also be ran as follows (relative to repo root):

```bash
source trends_and_insights_agent/.env

export RE_SA="service-${GOOGLE_CLOUD_PROJECT_NUMBER}@gcp-sa-aiplatform-re.iam.gserviceaccount.com"
gcloud secrets add-iam-policy-binding "projects/$GOOGLE_CLOUD_PROJECT/secrets/$YT_SECRET_MNGR_NAME" \
  --member="serviceAccount:$RE_SA" \
  --role="roles/secretmanager.secretAccessor"
```

In [21]:
import vertexai
from vertexai import agent_engines

GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT")
APP_NAME = "retail-data-agent"
BUCKET = os.getenv("BUCKET")

vertexai.init(
    project=GOOGLE_CLOUD_PROJECT,
    location="us-central1",
    staging_bucket=BUCKET,
)

remote_agent = agent_engines.create(
    agent_engine=my_agent,
    requirements=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
    ],
    extra_packages=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
    ],
)

Identified the following requirements: {'pydantic': '2.11.3', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.91.0'}


INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.3', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.91.0'}


Failed to parse constraint: deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl. Exception: Expected end or semicolon (after name and no valid version specifier)
    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


The following requirements are missing: {'pydantic', 'cloudpickle', 'google-cloud-aiplatform'}


...

Create AgentEngine backing LRO: projects/679926387543/locations/us-central1/reasoningEngines/8411677368858443776/operations/4644510391886413824


INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/679926387543/locations/us-central1/reasoningEngines/8411677368858443776/operations/4644510391886413824


View progress and logs at https://console.cloud.google.com/logs/query?project=wortz-project-352116


INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=wortz-project-352116


KeyboardInterrupt: 

### Save the agent engine resource name

This can be used by the command to deploy it to Agentspace

```bash
agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/1093257605637210112')
```

## 5. Try it remotely

In [ ]:
session1 = remote_agent.create_session(user_id="jwortz")
session1

{'events': [],
 'last_update_time': 1744661630.683849,
 'state': {},
 'id': '2567100715868618752',
 'app_name': '4697966894882226176',
 'user_id': 'jwortz'}

#### Same idea above applies to remote agents

In [ ]:
first_prompt = f"Check out the {EXAMPLE_BRIEF} and come up with some good campaign ideas. Be sure to generate a few image ideas for the campaign."
stream_agent(remote_agent, first_prompt)

# 6. Optional Cleanup

In [ ]:
remote_agent.delete(force=True)